# Assignment 1-1: Web Scraping

## Objective

Data scientists often need to crawl data from websites and turn the crawled data (HTML pages) to structured data (tables). Thus, web scraping is an essential skill that every data scientist should master. In this assignment, you will learn the followings:


* How to download HTML pages from a website?
* How to extract relevant content from an HTML page? 

Furthermore, you will gain a deeper understanding of the data science lifecycle.

**Requirements:**

1. Please use [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) rather than spark.DataFrame to manipulate data.

2. Please use [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) rather than [lxml](http://lxml.de/) to parse an HTML page and extract data from the page.

3. Please follow the python code style (https://www.python.org/dev/peps/pep-0008/). If TA finds your code hard to read, you will lose points. This requirement will stay for the whole semester.

## Preliminary

If this is your first time to write a web scraper, you need to learn some basic knowledge of this topic. I found that this is a good resource: [Tutorial: Web Scraping and BeautifulSoup](https://realpython.com/beautiful-soup-web-scraper-python/). 

Please let me know if you find a better resource. I'll share it with the other students.

## Overview

Imagine you are a data scientist working at SFU. Your job is to extract insights from SFU data to answer questions. 

In this assignment, you will do two tasks. Please recall the high-level data science lifecycle from Lecture 1. I suggest that when doing this assignment, please remind yourself of what data you collected and what questions you tried to answer.

## Task 1: SFU CS Faculty Members

Sometimes you don't know what questions to ask. No worries. Start collecting data first. 

In Task 1, your job is to write a web scraper to extract the faculty information from this page: [http://www.sfu.ca/computing/people/faculty.html](http://www.sfu.ca/computing/people/faculty.html).




### (a) Crawl Web Page

A web page is essentially a file stored in a remote machine (called web server). Please write code to download the HTML page and save it as a text file ("csfaculty.html").

In [ ]:
import requests

URL = 'http://www.sfu.ca/computing/people/faculty.html'
filename = 'csfaculty.html'

response = requests.get(URL)

if response.status_code == 200:
  # Response has succeeded
  html_content = response.text

  # Save the text to a file
  with open(filename, 'w', encoding='utf-8') as file:
    file.write(html_content)
  print(f'File {filename} successfully saved.')
else:
  print('Get response failed. Error code:', response.status_code)

### (b) Extract Structured Data

Please write code to extract relevant content (name, rank, area, profile, homepage) from "csfaculty.html" and save them as a CSV file (like [faculty_table.csv](./faculty_table.csv)). 

In [ ]:
from bs4 import BeautifulSoup as BS
import pandas as pd

with open(filename, 'r', encoding='utf-8') as file:
  html_content = file.read()

soup = BS(html_content, 'html.parser')

selector = '#page-content > section > div.main_content.parsys > div.parsys_column.cq-colctrl-lt0.people.faculty-list'

faculty_list = soup.select(selector)

data = []

for item in faculty_list:
  faculty_members = item.find_all(class_='textimage section')

  for member in faculty_members:
    # extract the content for each faulty member
    member_content = member.find('div', class_='text')

    # get the name and rank
    name_rank = member_content.find('h4').text.strip()
    name_rank = name_rank.split(',')
    name = name_rank[0].strip().title()
    if len(name_rank) > 1:
      rank = name_rank[1].split('\n')[0].strip().title()
    else:
      rank = None

    # get the area of specialty
    area_p_tag = member_content.find_all('p')
    area = area_p_tag[0].text.strip() if area_p_tag else 'p tag not found'
    area = area.replace('Area:', '').strip().title()

    # get both urls
    if len(area_p_tag) > 1:
      hrefs = [a.get('href') for a in area_p_tag[1].find_all('a') if a.get('href')]
    else:
      hrefs = [a.get('href') for a in area_p_tag[0].find_all('a') if a.get('href')]
    if len(hrefs) > 1 and hrefs[0] == hrefs[1]:
      profile = 'http://www.sfu.ca' + hrefs[0]
      homepage = 'http://www.sfu.ca' + hrefs[0]
    elif len(hrefs) == 1:
      profile = 'http://www.sfu.ca' + hrefs[0]
      homepage = None
    else:
      profile = 'http://www.sfu.ca' + hrefs[0]
      homepage = hrefs[1]

    member_data = [name, rank, area, profile, homepage]

    data.append(member_data)

headers = ['name', 'rank', 'area', 'profile', 'homepage']

# Create a pandas dataframe
df = pd.DataFrame(data, columns=headers)

# Save the dataframe to a file
outfilename = 'faculty_table.csv'
df.to_csv(outfilename)
print(f'Dataframe {outfilename} successfully saved.')

### (c) Interesting Finding

Note that you don't need to do anything for Task 1(c). The purpose of this part is to give you some sense about how to leverage exploratory data analysis (EDA) to come up with interesting questions about the data. EDA is an important topic in data science; you will  learn it soon from this course. 


First, please install [dataprep](http://dataprep.ai).
Then, run the cell below. 
It shows a bar chart for every column. What interesting findings can you get from these visualizations? 

In [ ]:
!pip install dataprep
from dataprep.eda import plot
import pandas as pd

df = pd.read_csv("faculty_table.csv")
plot(df)

Below are some examples:

**Finding 1:** Professor# (26) is more than 2x larger than Associate Professor# (10). 

**Questions:** Why did it happen? Is it common in all CS schools in Canada? Will the gap go larger or smaller in five years? What actions can be taken to enlarge/shrink the gap?


**Finding 2:** The Homepage has 22% missing values. 

**Questions:** Why are there so many missing values? Is it because many faculty do not have their own homepages or do not add their homepages to the school page? What actions can be taken to avoid this to happen in the future? 

## Task 2: Age Follows Normal Distribution?

In this task, you start with a question and then figure out what data to collect.

The question that you are interested in is `Does SFU CS faculty age follow a normal distribution?`

To estimate the age of a faculty member, you can collect the year in which s/he graduates from a university (`gradyear`) and then estimate `age` using the following equation:

$$age \approx 2021+23 - gradyear$$

For example, if one graduates from a university in 1990, then the age is estimated as 2021+23-1990 = 54. 



### (a) Crawl Web Page

You notice that faculty profile pages contain graduation information. For example, you can see that Dr. Jiannan Wang graduated from Harbin Institute of Technology in 2008 at [http://www.sfu.ca/computing/people/faculty/jiannanwang.html](http://www.sfu.ca/computing/people/faculty/jiannanwang.html). 


Please write code to download the 68 profile pages and save each page as a text file. 

In [ ]:
# Iterate over df['profile'] and save each page as a text file
import os
directory = 'profile_pages'
if not os.path.exists(directory):
  os.makedirs(directory)

# Use a loop to attempt multiple times in case the get request fails
from time import sleep

max_retries = 10
retry_delay = 2

for link in df['profile']:
  # Catch instances where http://www.sfu.ca has been appended twice to the str
  head = 'http://www.sfu.cahttp://www.sfu.ca'
  if link[0:len(head)] == head:
    link = link[len(head)//2:]

  for attempt in range(max_retries):
    try:
      response = requests.get(link)
      response.raise_for_status()  # Raises an HTTPError if the response was an HTTP error
      text = response.text
      filename = link
      filename = filename.split('/')[-1].split('.')[0].title() + '.txt'
      file_path = os.path.join(directory, filename)

      if os.path.exists(file_path):
        break # We already have this information

      with open(file_path, 'w', encoding='utf-8') as file:
        file.write(text)

      break  # Successful request, exit the loop

    except requests.RequestException as e:
      print(f'Attempt {attempt + 1} failed: {e}')
      sleep(retry_delay)
  else:
    print(f'All attempts failed for link {link}.')

### (b) Extract Structured Data

Please write code to extract the earliest graduation year (e.g., 2008 for Dr. Jiannan Wang) from each profile page, and create a csv file like [faculty_grad_year.csv](./faculty_grad_year.csv). 

In [ ]:
import re

headers = ['name', 'gradyear']

data = []

# iterate over each file in the directory
for filename in os.listdir(directory):

  # Check if it is a text file
  if filename.endswith('.txt'):
    file_path = os.path.join(directory, filename)

  # Open the file
  with open(file_path, 'r', encoding='utf-8') as file:
    soup = BS(file, 'html.parser')

    # Get the name
    title = soup.find('title').text.strip().title()
    name = title.split(' - ')[0]

    # Get the year
    divs = soup.find_all('div', class_='text parbase section')
    for div in divs:
      education_h2 = div.find(lambda tag: tag.name == 'h2' and tag.text.strip() == 'Education')
      if education_h2 is not None:
        text_list = [ele.text.strip() for ele in div if not None]
        for text in text_list:
          integers = re.findall(r'\d+', text)
          integers = [int(i) for i in integers if len(i) == 4]
          if len(integers) > 0:
            year = min(integers)
            row_data = [name, year]
            data.append(row_data)

name_year_df = pd.DataFrame(data, columns=(headers))
name_year_df.to_csv("faculty_grad_year.csv")

### (c) Interesting Finding

Similar to Task 1(c), you don't need to do anything here. Just look at different visualizations w.r.t. age and give yourself an answer to the question: `Does SFU CS faculty age follow a normal distribution?`

In [ ]:
from dataprep.eda import plot
import pandas as pd

df = pd.read_csv("faculty_grad_year.csv")
df["age"] = 2021+23-df["gradyear"]

plot(df, "age")

## Submission

Complete the code in this notebook, and submit it to the CourSys activity `Assignment 1`.